# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [40]:

def calcular_valor(numero, palo):
    
    valores_truco = {
        1: 14 if palo =="espada" else 13 if palo =="basto" else 8,  
        2: 9,  
        3: 10, 
        4: 1, 
        5: 2, 
        6: 3,
        7: 12 if palo == "espada" else 11 if palo == "oro" else 4, 
        10: 5, # 10
        11: 6, # 11
        12: 7  # 12
    }
    
    return valores_truco.get(numero, 0)  


class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)

    def __repr__(self):
        return f"{self.numero} de {self.palo} (Valor: {self.valor})"

    # Método para comparar dos cartas según su valor
    def __gt__(self, other):
        return self.valor > other.valor

    def __lt__(self, other):
        return self.valor < other.valor

    def __eq__(self, other):
        return self.valor == other.valor


    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)

    def __repr__(self):
        return f"{self.numero} de {self.palo} (Valor: {self.valor})"

    

c = Carta(1, "copa")
print(c)  


1 de copa (Valor: 8)


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [41]:
import random

class Mazo:
    def __init__(self):
        palos = ["oro","espada","copa","basto"]
        numeros =[1,2,3,4,5,6,7,10,11,12]
        self.cartas = [Carta(numero,palo)for numero in numeros for palo in palos]
        
    def mezclar(self):
        random.shuffle(self.cartas)

    def repartir(self, cantidad):
        if len(self.cartas) >= cantidad:
            return [self.cartas.pop() for _ in range(cantidad)]
        else:
            raise IndexError("No hay suficientes cartas en el mazo para repartir.")


m = Mazo()
m.mezclar()
print(m.repartir(3))

[12 de copa (Valor: 7), 7 de basto (Valor: 4), 5 de copa (Valor: 2)]


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [42]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.mano = []
        self.puntos = 0
    def recibir_cartas(self, cartas):
        self.mano = cartas
    
    def jugar_carta(self):
        if self.mano:
            carta = random.choice(self.mano)
            self.mano.remove(carta)
            return carta
        return None

j = Jugador("Juan")
j.recibir_cartas(m.repartir(3))
print(j.jugar_carta())

5 de espada (Valor: 2)


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [48]:
class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.mazo = Mazo()
        self.puntos_objetivo = 15

    def jugar_ronda(self):
        carta_j1 = self.jugador1.jugar_carta()
        carta_j2 = self.jugador2.jugar_carta()

        print(f"{self.jugador1.nombre} juega {carta_j1}")
        print(f"{self.jugador2.nombre} juega {carta_j2}")

        if carta_j1 > carta_j2:
            print(f"Gana la ronda {self.jugador1.nombre}")
            return self.jugador1
        elif carta_j1 < carta_j2:
            print(f"Gana la ronda {self.jugador2.nombre}")
            return self.jugador2
        else:
            print("Empate en la ronda")
            return None

    def jugar(self):
        while self.jugador1.puntos < self.puntos_objetivo and self.jugador2.puntos < self.puntos_objetivo:
            print("\n--- Nueva mano ---")
            self.mazo.mezclar()  # Mezclar el mazo

            try:
                self.jugador1.recibir_cartas(self.mazo.repartir(3))
                self.jugador2.recibir_cartas(self.mazo.repartir(3))
            except IndexError as e:
                print("Se terminaron las cartas en el mazo. Fin del juego.")
                break

            rondas_ganadas_j1 = 0
            rondas_ganadas_j2 = 0

            # Jugar 3 rondas
            for _ in range(3):
                ganador_ronda = self.jugar_ronda()
                if ganador_ronda == self.jugador1:
                    rondas_ganadas_j1 += 1
                elif ganador_ronda == self.jugador2:
                    rondas_ganadas_j2 += 1

            
            if rondas_ganadas_j1 > rondas_ganadas_j2:
                print(f"Ganador de la mano: {self.jugador1.nombre}")
                self.jugador1.puntos += 1
            elif rondas_ganadas_j2 > rondas_ganadas_j1:
                print(f"Ganador de la mano: {self.jugador2.nombre}")
                self.jugador2.puntos += 1
            else:
                print("La mano terminó en empate. No se suman puntos.")

            
            print(f"Puntaje: {self.jugador1.nombre} {self.jugador1.puntos} - {self.jugador2.puntos} {self.jugador2.nombre}")


        if self.jugador1.puntos >= self.puntos_objetivo:
            print(f"\n¡{self.jugador1.nombre} ha ganado la partida!")
        elif self.jugador2.puntos >= self.puntos_objetivo:
            print(f"\n¡{self.jugador2.nombre} ha ganado la partida!")
        else:
            print("\nEl juego ha terminado sin un ganador definitivo debido a la falta de cartas.")


p = Partida(Jugador("Juan"), Jugador("Pedro"))
p.jugar()


--- Nueva mano ---
Juan juega 10 de espada (Valor: 5)
Pedro juega 11 de oro (Valor: 6)
Gana la ronda Pedro
Juan juega 12 de basto (Valor: 7)
Pedro juega 10 de copa (Valor: 5)
Gana la ronda Juan
Juan juega 3 de espada (Valor: 10)
Pedro juega 6 de basto (Valor: 3)
Gana la ronda Juan
Ganador de la mano: Juan
Puntaje: Juan 1 - 0 Pedro

--- Nueva mano ---
Juan juega 4 de copa (Valor: 1)
Pedro juega 10 de oro (Valor: 5)
Gana la ronda Pedro
Juan juega 12 de espada (Valor: 7)
Pedro juega 5 de oro (Valor: 2)
Gana la ronda Juan
Juan juega 4 de espada (Valor: 1)
Pedro juega 3 de basto (Valor: 10)
Gana la ronda Pedro
Ganador de la mano: Pedro
Puntaje: Juan 1 - 1 Pedro

--- Nueva mano ---
Juan juega 7 de oro (Valor: 11)
Pedro juega 12 de copa (Valor: 7)
Gana la ronda Juan
Juan juega 5 de basto (Valor: 2)
Pedro juega 10 de basto (Valor: 5)
Gana la ronda Pedro
Juan juega 3 de oro (Valor: 10)
Pedro juega 6 de espada (Valor: 3)
Gana la ronda Juan
Ganador de la mano: Juan
Puntaje: Juan 2 - 1 Pedro

---